In [1]:
# 1. Cài đặt các thư viện cần thiết
!pip install -q transformers datasets evaluate sentencepiece accelerate google-generativeai

import os
import shutil
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer
)
import evaluate
import numpy as np
from datasets import load_from_disk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 35.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2025-11-29 11:17:14.642432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764415034.979072      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764415035.076713      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# 1. TÌM THƯ MỤC GỐC (Nơi chứa dataset_dict.json)
INPUT_DIR = "/kaggle/input"
dataset_root_path = None

print("🔍 Đang tìm file 'dataset_dict.json'...")

for root, dirs, files in os.walk(INPUT_DIR):
    if "dataset_dict.json" in files:
        dataset_root_path = root
        print(f"✅ Đã tìm thấy gốc dataset tại: {dataset_root_path}")
        break

if not dataset_root_path:
    raise FileNotFoundError("❌ Không tìm thấy file dataset_dict.json! Hãy kiểm tra lại Input.")

# 2. COPY SANG WORKING (Bắt buộc để tránh lỗi Read-only)
# HuggingFace cần ghi file cache/lock, nên không thể load trực tiếp từ Input
WORKING_DATA_PATH = "/kaggle/working/data_ready"

if os.path.exists(WORKING_DATA_PATH):
    print(f"ℹ️ Thư mục {WORKING_DATA_PATH} đã tồn tại, đang xóa để copy mới...")
    shutil.rmtree(WORKING_DATA_PATH)

print(f"🔄 Đang copy dataset sang {WORKING_DATA_PATH}...")
shutil.copytree(dataset_root_path, WORKING_DATA_PATH)
print("✅ Copy hoàn tất!")

# 3. LOAD DATASET
print("📥 Đang load dataset...")
datasets = load_from_disk(WORKING_DATA_PATH)

print("\n📊 CẤU TRÚC DỮ LIỆU ĐÃ LOAD:")
print(datasets)

# Kiểm tra số lượng để em yên tâm
if 'train' in datasets:
    print(f"   - Train: {len(datasets['train'])} dòng")
if 'test' in datasets:
    print(f"   - Test:  {len(datasets['test'])} dòng")
if 'validation' in datasets:
    print(f"   - Val:   {len(datasets['validation'])} dòng")

🔍 Đang tìm file 'dataset_dict.json'...
✅ Đã tìm thấy gốc dataset tại: /kaggle/input/cleaned-dataset
🔄 Đang copy dataset sang /kaggle/working/data_ready...
✅ Copy hoàn tất!
📥 Đang load dataset...

📊 CẤU TRÚC DỮ LIỆU ĐÃ LOAD:
DatasetDict({
    train: Dataset({
        features: ['en_text', 'vi_text'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['en_text', 'vi_text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['en_text', 'vi_text'],
        num_rows: 1000
    })
})
   - Train: 10000 dòng
   - Test:  1000 dòng
   - Val:   1000 dòng


In [3]:
print(datasets['test'][:10])

{'en_text': ["Team 's playing Bethesda Medical next week .\n", "The latest I 've read , seen , thought is that it 's incredibly hard for them to do that .\n", 'And so this had me stymied for quite a long period of time .\n', 'Can you imagine why ?\n', 'Get this horse off of me .\n', 'Is this some sort of joke to you ?\n', 'The team had a system .\n', 'I spent months educating nurses and doctors about what female genital mutilation was and where it was practiced : Africa , the Middle East , Asia , and now , Australia and London and America , because , as we all know , we live in a multicultural society , and people who come from those backgrounds come with their culture , and sometimes they have cultural practices that we may not agree with , but they continue to practice them .\n', 'Right through that door .\n', 'Presented to me by Heracleo himself .\n'], 'vi_text': ['Tuần sau tham dự giải bóng chày Bethesda Medical .\n', 'Theo những gì mới đây nhất tôi đã đọc , xem và suy nghĩ thì điề

In [6]:
# Chuyển 10 dòng đầu sang bảng để dễ nhìn
print(datasets['test'].select(range(50)).to_pandas())

                                              en_text  \
0      Team 's playing Bethesda Medical next week .\n   
1   The latest I 've read , seen , thought is that...   
2   And so this had me stymied for quite a long pe...   
3                             Can you imagine why ?\n   
4                        Get this horse off of me .\n   
5                Is this some sort of joke to you ?\n   
6                           The team had a system .\n   
7   I spent months educating nurses and doctors ab...   
8                         Right through that door .\n   
9             Presented to me by Heracleo himself .\n   
10   Also keep in mind that people want to hear wh...   
11  He also led Sanna Khanh Hoa FC ( 2001 ) and Ha...   
12  But suppose the engineer 's name ai n't Manuel...   
13  I was on my way up to x -ray and I thought I '...   
14             Keep the seeds indoors as they dry .\n   
15  It was announced on Friday January 11th , 2018...   
16                    He 'll ne

In [7]:
!pip install -q pandas google-generativeai

import google.generativeai as genai
import pandas as pd
import json
import time
import os

In [10]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# --- 1. CẤU HÌNH API ---
YOUR_API_KEY = ""
genai.configure(api_key=YOUR_API_KEY)

model = genai.GenerativeModel('gemini-flash-latest')

# --- 2. BỘ PROMPT "THUỐC ĐẶC TRỊ HALLUCINATION" ---
PROMPTS = {
    # 1. BIẾN THỂ SỐ LIỆU (500 câu)
    "hallu_numbers": """
    Bạn là chuyên gia kiểm toán dữ liệu. Hãy tạo {batch_size} cặp câu Anh-Việt chứa DỮ LIỆU ĐỊNH LƯỢNG DÀY ĐẶC.
    
    Yêu cầu bắt buộc:
    1. Mỗi câu PHẢI chứa ít nhất 2 thông số cụ thể: Ngày tháng, Giờ giấc, Tiền tệ ($), Số đo (kg, km), Tỉ lệ (%).
    2. Sử dụng số liệu phức tạp: "1,200.50", "3.5 billion", "the late 1990s", "0.05%".
    3. Bản dịch tiếng Việt phải **chính xác tuyệt đối từng con số**, KHÔNG làm tròn, KHÔNG đổi đơn vị.
    
    Trả về định dạng JSON List: [{{"en": "Revenue rose by 5.2% to $3M in Q4.", "vi": "Doanh thu tăng 5,2% lên 3 triệu đô la trong Quý 4."}}]
    """,
    
    # 2. BIẾN THỂ TÊN RIÊNG (500 câu)
    "hallu_names": """
    Bạn là biên dịch viên thời sự. Hãy tạo {batch_size} cặp câu Anh-Việt chứa nhiều TÊN RIÊNG (Named Entities).
    
    Yêu cầu bắt buộc:
    1. Chứa **Tên người nước ngoài, Tên công ty, Tên tổ chức, Tên địa danh lạ** (VD: "Mr. Armstrong", "Pfizer", "WHO", "Novosibirsk").
    2. Chứa các Mã hiệu/Ký hiệu: "Model X-200", "Apollo 11", "COVID-19".
    3. Bản dịch tiếng Việt: Giữ nguyên tên riêng (hoặc phiên âm chuẩn), TUYỆT ĐỐI KHÔNG dịch tên người sang nghĩa đen.
    
    Trả về định dạng JSON List: [{{"en": "Mr. Stone visited Little Rock yesterday.", "vi": "Ông Stone đã đến thăm Little Rock hôm qua."}}]
    """,
    
    # 3. BIẾN THỂ DỊCH SÁT NGHĨA (500 câu)
    "hallu_strict": """
    Bạn là kỹ sư viết hướng dẫn kỹ thuật. Hãy tạo {batch_size} cặp câu Anh-Việt dạng CÂU LỆNH hoặc QUY TRÌNH NGHIÊM NGẶT.
    
    Yêu cầu bắt buộc:
    1. Câu ngắn, trực diện, dùng động từ mạnh: "Press", "Hold", "Cut", "Stop".
    2. Nội dung khô khan, logic: "If the light is red, do not open."
    3. Bản dịch tiếng Việt: **Dịch word-by-word (bám sát nghĩa đen)**, KHÔNG thêm từ đệm, KHÔNG sáng tạo cảm xúc, KHÔNG diễn giải dài dòng.
    
    Trả về định dạng JSON List: [{{"en": "Push the red button to emergency stop.", "vi": "Nhấn nút đỏ để dừng khẩn cấp."}}]
    """
}

# --- 3. HÀM SINH DỮ LIỆU (Đã tối ưu Safety & JSON) ---
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}

def generate_data(topic, target_count=500, batch_size=20): # Mặc định 20 cho an toàn
    print(f"\n🚀 Đang sinh dữ liệu chủ đề: {topic.upper()}...")
    generated_data = []
    # Tính toán số batch cần chạy
    num_batches = (target_count + batch_size - 1) // batch_size 
    
    for i in range(num_batches):
        print(f"   - Batch {i+1}/{num_batches}...", end="")
        
        prompt = PROMPTS[topic].format(batch_size=batch_size)
        
        try:
            response = model.generate_content(prompt, safety_settings=safety_settings)
            
            # Kiểm tra chặn
            if not response.parts:
                print(f" ⚠️ Bị chặn! Lý do: {response.prompt_feedback}")
                continue
            
            # Xử lý JSON thông minh
            clean_text = response.text.strip()
            start_idx = clean_text.find('[')
            end_idx = clean_text.rfind(']') + 1
            
            if start_idx != -1 and end_idx != -1:
                clean_text = clean_text[start_idx:end_idx]
                batch_items = json.loads(clean_text)
                generated_data.extend(batch_items)
                print(f" ✅ Xong ({len(batch_items)} câu).")
            else:
                print(" ❌ Lỗi format JSON.")
                
        except Exception as e:
            print(f" ❌ Lỗi: {e}. Bỏ qua.")
        
        time.sleep(2) 
        
    return generated_data

# --- 4. CHẠY THỰC TẾ ---
all_synthetic_data = []

# Danh sách các chủ đề mới (Khớp với key trong PROMPTS)
list_topics = ["hallu_numbers", "hallu_names", "hallu_strict"]

for topic in list_topics:
    # Mỗi chủ đề sinh 500 câu, batch 20
    data = generate_data(topic, target_count=500, batch_size=20)
    for item in data:
        item['type'] = topic
    all_synthetic_data.extend(data)

# --- 5. LƯU FILE ---
if all_synthetic_data:
    df_syn = pd.DataFrame(all_synthetic_data)
    df_syn = df_syn.rename(columns={"en": "en_text", "vi": "vi_text"})
    
    filename = "synthetic_hallucination_fix.csv"
    df_syn.to_csv(filename, index=False)
    print(f"\n🎉 TỔNG KẾT: Đã sinh được {len(df_syn)} câu.")
    print(df_syn.groupby('type').count())
else:
    print("❌ Không có dữ liệu nào được sinh ra.")


🚀 Đang sinh dữ liệu chủ đề: HALLU_NUMBERS...
   - Batch 1/25... ✅ Xong (20 câu).
   - Batch 2/25... ✅ Xong (20 câu).
   - Batch 3/25... ✅ Xong (20 câu).
   - Batch 4/25... ✅ Xong (20 câu).
   - Batch 5/25... ✅ Xong (20 câu).
   - Batch 6/25... ✅ Xong (20 câu).
   - Batch 7/25... ✅ Xong (20 câu).
   - Batch 8/25... ✅ Xong (20 câu).
   - Batch 9/25... ✅ Xong (20 câu).
   - Batch 10/25... ✅ Xong (20 câu).
   - Batch 11/25... ✅ Xong (20 câu).
   - Batch 12/25... ✅ Xong (20 câu).
   - Batch 13/25... ✅ Xong (20 câu).
   - Batch 14/25... ✅ Xong (20 câu).
   - Batch 15/25... ✅ Xong (20 câu).
   - Batch 16/25... ✅ Xong (20 câu).
   - Batch 17/25... ✅ Xong (20 câu).
   - Batch 18/25... ✅ Xong (20 câu).
   - Batch 19/25... ✅ Xong (20 câu).
   - Batch 20/25... ✅ Xong (20 câu).
   - Batch 21/25... ✅ Xong (20 câu).
   - Batch 22/25... ✅ Xong (20 câu).
   - Batch 23/25... ✅ Xong (20 câu).
   - Batch 24/25... ✅ Xong (20 câu).
   - Batch 25/25... ✅ Xong (20 câu).

🚀 Đang sinh dữ liệu chủ đề: HALLU_NAM

In [11]:
def clean_data(df):
    print("\n🧹 Đang làm sạch dữ liệu...")
    initial_len = len(df)
    
    # Xóa dòng Null
    df = df.dropna(subset=['en_text', 'vi_text'])
    
    # Xóa dòng trùng lặp (Duplicate)
    df = df.drop_duplicates(subset=['en_text'])
    
    # Tính độ dài số từ
    df['len_en'] = df['en_text'].astype(str).apply(lambda x: len(x.split()))
    df['len_vi'] = df['vi_text'].astype(str).apply(lambda x: len(x.split()))
    
    # Lọc câu quá ngắn (rác) hoặc quá dài (gây OOM)
    # Giữ lại câu từ 3 đến 150 từ
    df = df[(df['len_en'] >= 3) & (df['len_en'] <= 150)]
    df = df[(df['len_vi'] >= 3) & (df['len_vi'] <= 150)]
    
    print(f"   - Ban đầu: {initial_len:,} dòng")
    print(f"   - Còn lại: {len(df):,} dòng sạch")
    return df

df_clean = clean_data(df_syn)


🧹 Đang làm sạch dữ liệu...
   - Ban đầu: 1,500 dòng
   - Còn lại: 1,487 dòng sạch


In [12]:
from datasets import Dataset, concatenate_datasets

synthetic_ds = Dataset.from_pandas(df_clean, preserve_index=False)
print(f"✅ Đã chuyển đổi synthetic data: {len(synthetic_ds)} dòng")

✅ Đã chuyển đổi synthetic data: 1487 dòng


In [13]:
# --- 2. KIỂM TRA TÊN CỘT (BẮT BUỘC) ---
# Hai tập phải có tên cột Y HỆT nhau mới gộp được
print(f"🔍 Cột Dataset gốc: {datasets['train'].column_names}")
print(f"🔍 Cột Synthetic:   {synthetic_ds.column_names}")

# Nếu tập gốc đang dùng tên khác (vd 'en', 'vi'), ta đổi synthetic theo nó
# Hoặc đổi cả 2 về 'en_text', 'vi_text' (Khuyên dùng)
if datasets['train'].column_names != synthetic_ds.column_names:
    print("⚠️ Tên cột chưa khớp! Đang tự động đồng bộ...")
    # Ví dụ: Chỉ giữ lại 2 cột chính để tránh lỗi lệch cột 'id' hay 'index'
    cols_to_keep = ['en_text', 'vi_text']
    datasets['train'] = datasets['train'].select_columns(cols_to_keep)
    synthetic_ds = synthetic_ds.select_columns(cols_to_keep)

🔍 Cột Dataset gốc: ['en_text', 'vi_text']
🔍 Cột Synthetic:   ['en_text', 'vi_text', 'type', 'len_en', 'len_vi']
⚠️ Tên cột chưa khớp! Đang tự động đồng bộ...


In [14]:
# Danh sách các cột cần giữ lại (cho khớp với tập gốc)
cols_to_keep = ['en_text', 'vi_text']

print(f"✂️ Đang cắt bỏ cột thừa trong Synthetic Data...")
print(f"   - Trước: {synthetic_ds.column_names}")

# Chỉ chọn đúng 2 cột cần thiết
synthetic_ds = synthetic_ds.select_columns(cols_to_keep)

# (Cẩn thận hơn) Áp dụng luôn cho tập train gốc để đảm bảo 100% khớp
datasets['train'] = datasets['train'].select_columns(cols_to_keep)

print(f"   - Sau:   {synthetic_ds.column_names}")

# --- KIỂM TRA LẠI ---
if synthetic_ds.column_names == datasets['train'].column_names:
    print("✅ Hai tập đã khớp cột hoàn toàn! Sẵn sàng gộp.")
else:
    print("❌ Vẫn chưa khớp. Hãy kiểm tra lại tên cột.")

✂️ Đang cắt bỏ cột thừa trong Synthetic Data...
   - Trước: ['en_text', 'vi_text']
   - Sau:   ['en_text', 'vi_text']
✅ Hai tập đã khớp cột hoàn toàn! Sẵn sàng gộp.


In [15]:
# --- 3. GỘP VÀ XÁO TRỘN ---
print(f"\n🔄 Đang gộp: {len(datasets['train'])} (Gốc) + {len(synthetic_ds)} (Synthetic)...")

# Gộp
combined_train = concatenate_datasets([datasets['train'], synthetic_ds])

# Xáo trộn (Shuffle) để data mới và cũ trộn đều vào nhau
combined_train = combined_train.shuffle(seed=42)

# Cập nhật lại vào biến datasets chính
datasets['train'] = combined_train

# --- KẾT QUẢ ---
print("\n🎉 HOÀN TẤT GỘP DỮ LIỆU THÔ!")
print(f"📊 Tổng tập Train mới: {len(datasets['train'])} dòng")
print(datasets)
print("🔍 Mẫu dữ liệu ngẫu nhiên:", datasets['train'][0])


🔄 Đang gộp: 10000 (Gốc) + 1487 (Synthetic)...

🎉 HOÀN TẤT GỘP DỮ LIỆU THÔ!
📊 Tổng tập Train mới: 11487 dòng
DatasetDict({
    train: Dataset({
        features: ['en_text', 'vi_text'],
        num_rows: 11487
    })
    validation: Dataset({
        features: ['en_text', 'vi_text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['en_text', 'vi_text'],
        num_rows: 1000
    })
})
🔍 Mẫu dữ liệu ngẫu nhiên: {'en_text': "It 's -- it 's fine .\n", 'vi_text': '- Không sao đâu .\n'}


In [18]:
from transformers import AutoTokenizer
from datasets import concatenate_datasets, DatasetDict
from IPython.display import FileLink

# --- 1. CẤU HÌNH ---
MODEL_CHECKPOINT = "VietAI/vit5-base"
MAX_LENGTH = 128

# Định nghĩa 2 loại prefix
PREFIX_EN_VI = "dịch Anh sang Việt: "
PREFIX_VI_EN = "dịch Việt sang Anh: "

print(f"⚙️ Đang tải Tokenizer: {MODEL_CHECKPOINT}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# --- 2. HÀM TOKENIZE THÔNG MINH (2 CHIỀU) ---
def preprocess_bidirectional(examples):
    # --- CHIỀU 1: EN -> VI ---
    inputs_en_vi = [PREFIX_EN_VI + str(ex) for ex in examples["en_text"]]
    targets_en_vi = [str(ex) for ex in examples["vi_text"]]
    
    # --- CHIỀU 2: VI -> EN (Đảo ngược lại) ---
    inputs_vi_en = [PREFIX_VI_EN + str(ex) for ex in examples["vi_text"]]
    targets_vi_en = [str(ex) for ex in examples["en_text"]]
    
    # Gộp cả 2 chiều vào chung 1 list
    # Kết quả sẽ là: [En->Vi sample 1, Vi->En sample 1, En->Vi sample 2, ...]
    all_inputs = inputs_en_vi + inputs_vi_en
    all_targets = targets_en_vi + targets_vi_en
    
    # Tokenize toàn bộ
    model_inputs = tokenizer(
        all_inputs, 
        text_target=all_targets, 
        max_length=MAX_LENGTH, 
        truncation=True
    )
    return model_inputs

# --- 3. THỰC HIỆN ---
# Giả sử em đang có biến 'datasets' chứa 3 tập train/test/val (dạng text thô)
# Lưu ý: Lúc này số lượng mẫu sẽ TĂNG GẤP ĐÔI

print("🔄 Đang Tokenize dữ liệu 2 chiều (Bidirectional)...")

# Xóa các cột cũ để tránh lỗi (vì giờ input/target đã trộn lẫn lộn)
column_names = datasets["train"].column_names

tokenized_datasets = datasets.map(
    preprocess_bidirectional,
    batched=True,
    remove_columns=column_names,
    desc="Running bidirectional tokenizer"
)

# Shuffle (Xáo trộn) lại lần nữa để model không học theo cụm
# (Tránh trường hợp model học hết En-Vi rồi mới học Vi-En)
tokenized_datasets = tokenized_datasets.shuffle(seed=42)

# --- 4. KIỂM TRA KẾT QUẢ ---
print("\n✅ Tokenize 2 chiều hoàn tất!")
print(tokenized_datasets)

print("\n🔍 Soi thử dữ liệu (Em sẽ thấy lúc thì input là tiếng Anh, lúc là tiếng Việt):")
for i in range(3):
    sample = tokenized_datasets['train'][i]
    print(f"Sample {i}:")
    print(f" - Input: {tokenizer.decode(sample['input_ids'], skip_special_tokens=True)}")
    print(f" - Label: {tokenizer.decode(sample['labels'], skip_special_tokens=True)}")
    print("-" * 30)

# --- 5. LƯU VÀ NÉN ---
SAVE_PATH = "/kaggle/working/tokenized_sync_data"
ZIP_NAME = "dataset_sync_tokenized"

print(f"\n💾 Đang lưu dataset đã xử lý vào {SAVE_PATH}...")
tokenized_datasets.save_to_disk(SAVE_PATH)

print(f"📦 Đang nén thành {ZIP_NAME}.zip ...")
shutil.make_archive(f"/kaggle/working/{ZIP_NAME}", 'zip', SAVE_PATH)

# --- 6. TẠO LINK TẢI ---
print("\n🎉 XONG TOÀN BỘ GIAI ĐOẠN 1!")
print("👇 Bấm vào link dưới đây để tải dataset 'ăn liền' về máy tính:")
display(FileLink(f"{ZIP_NAME}.zip"))

⚙️ Đang tải Tokenizer: VietAI/vit5-base...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/22974 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

📦 Đang nén thành dataset_sync_tokenized.zip ...

🎉 XONG TOÀN BỘ GIAI ĐOẠN 1!
👇 Bấm vào link dưới đây để tải dataset 'ăn liền' về máy tính:


/kaggle/working/dataset_sync_tokenized.zip